WINDOW SPLIT

In [ ]:
from src.execution import WindowSplittingExecution

In [ ]:
DATA: str = "assets/data/real-data.csv"
VALIDATION_SPLIT: bool = True # If the data should also be splitted into a validation set?
TEST_SIZE: float = 0.6
BATCH_SPLIT: bool = True
BATCHSIZE: int = 1800 # real-data: 1800, synthetic-data: 326
SEED: int = 3748 # Seed for random state -> Split with same seed and data will always result in the same split
INTERPOLATION: bool = False
WINDOWSIZE: int = 10
SEP: str = ";" # Separator of the csv file
DECIMAL: str = "," # Decimal separator of the csv file


WindowSplittingExecution.execute(DATA, 
                                 BATCH_SPLIT, 
                                 VALIDATION_SPLIT, 
                                 TEST_SIZE, 
                                 SEED, 
                                 BATCHSIZE, 
                                 INTERPOLATION, 
                                 WINDOWSIZE, 
                                 SEP, 
                                 DECIMAL)

TUNE MODEL

In [ ]:
from src.execution import CNNTuningExecution

In [ ]:
SPLITTED_DATA_FOLDER = "build/window_split/sim_data_preprocessed_offset_multiplitiv_with_rot_forces/1743718326"

CNNTuningExecution.execute(SPLITTED_DATA_FOLDER)

TRAIN MODEL

In [ ]:
from src.execution import CNNTrainingExecution

In [ ]:
TUNED_MODEL_FILE = "/home/davi/Documents/adp-machine-learning/build/tune/real-data/1743693239/real.h5"
DATA_FOLDER = "build/window_split/real-data/1743522733"
SAVE_FILENMAME = "cnn-20-real.h5" # With file extension
LEARNING_RATE = 0.0009248893590788498 # !!!!

CNNTrainingExecution.execute(TUNED_MODEL_FILE, DATA_FOLDER, SAVE_FILENMAME, LEARNING_RATE)

VALIDATE MODEL

In [ ]:
from src.execution import CNNValidationExecution 

In [ ]:
MODEL_FOLDER = "build/soft-start/cnn/1743863962"
DATA_FOLDER = "build/window_split/real-data/1743522733"

CNNValidationExecution.execute(MODEL_FOLDER, DATA_FOLDER)

In [ ]:
from src.execution import CNNSoftStartExecution

In [ ]:
MODEL_FILE = "build/train/cnn-synthetic-data-offset/1742244525/cnn-synthetic-offset.h5"
DATA_FOLDER = "build/window_split/real-data/1742552872"
SAVE_FILENAME = "cnn-retrained.h5"
LEARNING_RATE = 0.000029335291375872573

CNNSoftStartExecution.execute(MODEL_FILE, DATA_FOLDER, SAVE_FILENAME, LEARNING_RATE)

In [ ]:
from src.experiment import MultipleCNNTrainingExperiment

In [ ]:
MultipleCNNTrainingExperiment.start(N=10, 
                                    test_size=0.2, 
                                    model_file="build/tune/real-data/1743693239/best_model.h5", 
                                    data_file="assets/data/real-data.csv",
                                    learning_rate=0.0009248893590788498, 
                                    generate_new_split=True,
                                    sep=";",
                                    decimal=",", 
                                    batchsize=1800, 
                                    batch_split=True)

In [ ]:
from src.experiment import SoftStartExperiment

In [ ]:
SoftStartExperiment.run(model_file="build/train/syn-off-mul/1743673354/cnn.h5", 
                        data_folder="build/window_split/real-data/1743939075",
                        lr_factors=[0.1],
                        unfreezed_layers=[3, 4, 6, 7, 8, 12 ])

In [ ]:
from src.experiment import MultipleCNNValidationExperiment

In [ ]:
MODEL_FOLDER = "build/soft-start/var-layers-80"
N = 7
MAKE_VALIDATION = True
DATA_FOLDER = None

MultipleCNNValidationExperiment.start(MODEL_FOLDER, N, MAKE_VALIDATION, DATA_FOLDER)

In [ ]:
from src.execution import CNNValidationExecution

In [ ]:
import os

DATA_FOLDER = "build/window_split/real-data"
N = 10

for data_folder in DATA_FOLDER:
    folders = sorted([f.path for f in os.scandir(DATA_FOLDER) if f.is_dir()])

for data_folder in folders[-N:]:
    CNNValidationExecution.execute(model_folder="assets/models/corvin/0", 
                                   data_folder=data_folder)


In [ ]:
import keras

In [ ]:
m =  keras.models.load_model("build/tune/sim_data_preprocessed_offset_multiplitiv/1743667824/best-model.h5")
print(m.summary())